In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from matpower import start_instance

from matpowercaseframes import CaseFrames

m = start_instance()

In [3]:
CASE_NAME = "case9"

mpc = m.loadcase(CASE_NAME, verbose=False)
cf_mpc = CaseFrames(mpc)  # _read_oct2py_struct
cf_parse = CaseFrames(CASE_NAME)  # _read_matpower

In [4]:
for attribute in cf_mpc.attributes:
    df_mpc = getattr(cf_mpc, attribute)
    df_parse = getattr(cf_parse, attribute)

    if isinstance(df_mpc, pd.DataFrame):
        print(df_mpc.columns.equals(df_parse.columns))
        print(f"{attribute}: {df_mpc.equals(df_parse)}")
    else:
        print(f"{attribute}: {df_mpc == df_parse}")

version: True
baseMVA: True
True
bus: False
True
gen: False
True
branch: False
True
gencost: False


In [5]:
cf_parse.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1,72.3,27.03,300,-300,1.040,100,1,250,10,...,0,0,0,0,0,0,0,0,0,0
2,2,163.0,6.54,300,-300,1.025,100,1,300,10,...,0,0,0,0,0,0,0,0,0,0
3,3,85.0,-10.95,300,-300,1.025,100,1,270,10,...,0,0,0,0,0,0,0,0,0,0


In [6]:
def astype_numpy(df):
    # 1.14 ms ± 26.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
    df = df.convert_dtypes()

    columns = df.select_dtypes(include=["integer"]).columns
    df[columns] = df[columns].astype(int, errors="ignore")

    columns = df.select_dtypes(include=["float"]).columns
    df[columns] = df[columns].astype(float, errors="ignore")

    columns = df.select_dtypes(include=["string"]).columns
    df[columns] = df[columns].astype(str)

    columns = df.select_dtypes(include=["boolean"]).columns
    df[columns] = df[columns].astype(bool)

    return df


# def astype_numpy2(df):
#     # 1.61 ms ± 41.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
#     # Convert to pandas best types first
#     df = df.convert_dtypes()

#     # Batch conversion for each dtype group to minimize indexing
#     conversions = {"integer": int, "float": float, "string": str, "boolean": bool}

#     for dtype, target_type in conversions.items():
#         columns = df.select_dtypes(include=[dtype]).columns
#         if not columns.empty:  # Only attempt conversion if columns exist
#             df[columns] = df[columns].apply(
#                 lambda col: col.astype(target_type), axis=0
#             )

#     return df

In [7]:
cf_mpc.gen["PG"].dtypes, astype_numpy(cf_mpc.gen)["PG"].dtypes

(dtype('<f8'), dtype('float64'))

In [8]:
cf_mpc.gen.dtypes, astype_numpy(cf_mpc.gen).dtypes

(GEN_BUS       float64
 PG            float64
 QG            float64
 QMAX          float64
 QMIN          float64
 VG            float64
 MBASE         float64
 GEN_STATUS    float64
 PMAX          float64
 PMIN          float64
 PC1           float64
 PC2           float64
 QC1MIN        float64
 QC1MAX        float64
 QC2MIN        float64
 QC2MAX        float64
 RAMP_AGC      float64
 RAMP_10       float64
 RAMP_30       float64
 RAMP_Q        float64
 APF           float64
 dtype: object,
 GEN_BUS         int64
 PG            float64
 QG            float64
 QMAX            int64
 QMIN            int64
 VG            float64
 MBASE           int64
 GEN_STATUS      int64
 PMAX            int64
 PMIN            int64
 PC1             int64
 PC2             int64
 QC1MIN          int64
 QC1MAX          int64
 QC2MIN          int64
 QC2MAX          int64
 RAMP_AGC        int64
 RAMP_10         int64
 RAMP_30         int64
 RAMP_Q          int64
 APF             int64
 dtype: object)

In [9]:
cf_mpc.infer_numpy()  # introduced in 1.0.10
cf_mpc.gen.dtypes

GEN_BUS         int64
PG            float64
QG            float64
QMAX            int64
QMIN            int64
VG            float64
MBASE           int64
GEN_STATUS      int64
PMAX            int64
PMIN            int64
PC1             int64
PC2             int64
QC1MIN          int64
QC1MAX          int64
QC2MIN          int64
QC2MAX          int64
RAMP_AGC        int64
RAMP_10         int64
RAMP_30         int64
RAMP_Q          int64
APF             int64
dtype: object